In [1]:
import ROOT
import re
import numpy as np

def load_event_class(filename):
    """Read a C++ root class definition, generating dictionaries for vectors of classes"""
    classnames = []
    with open(filename, 'r') as classfile:
        for line in classfile.readlines():
            m = re.match(r'class (\w*) ', line)
            if m:
                classnames.append(m.group(1))
# Load the file in ROOT
    ROOT.gROOT.ProcessLine('.L %s++' % filename)
# Build the required dictionaries for the vectors of classes
    for name in classnames:
        ROOT.gInterpreter.GenerateDictionary("vector<%s>" % name, filename)

In [2]:
load_event_class("pax_event_class_410.cpp")

TClassTable::Add:0: RuntimeWarning: class Interaction already in TClassTable
TClassTable::Add:0: RuntimeWarning: class ReconstructedPosition already in TClassTable
TClassTable::Add:0: RuntimeWarning: class Peak already in TClassTable
TClassTable::Add:0: RuntimeWarning: class Event already in TClassTable


In [3]:
from sys import getrefcount
import numpy as np
root_event = ROOT.Event()

In [4]:
root_field = getattr(root_event, "lone_hits_per_channel") # an array type of field

In [5]:
type(root_field)

ROOT.PyShortBuffer

In [6]:
getrefcount(root_field)

2

In [7]:
lone_hits_per_channel = np.zeros(243, dtype=np.int16) # the "field_value" that we convert

In [8]:
for i, x in enumerate(lone_hits_per_channel):
    root_field[i] = x

In [9]:
getrefcount(root_field) # huge number of references

488

In [11]:
# how about this instead:
import array

root_field = getattr(root_event, "lone_hits_per_channel")    
root_field_type = root_field.typecode.decode("UTF-8")
root_field = array.array(root_field_type,
                                         [0]*lone_hits_per_channel.size)


In [12]:
for i, x in enumerate(lone_hits_per_channel):
    root_field[i] = x

In [14]:
getrefcount(root_field) # no refs?

2

In [15]:
setattr(root_event, "lone_hits_per_channel", root_field)